In [37]:
!pip install -q google-genai google-adk

In [38]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [39]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [40]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [41]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [42]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

# Editada: Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

import asyncio  # Importar para usar asyncio.sleep
from google.api_core import exceptions  # Importar para capturar exceções da API

async def call_agent(agent: Agent, message_text: str) -> str:
    # Número máximo de tentativas
    max_retries = 5
    # Atraso inicial em segundos
    initial_delay = 1

    for attempt in range(max_retries):
        try:
            # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
            session = await session_service.create_session(app_name=agent.name, user_id="user1")
            # Cria um Runner para o agente
            runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
            # Cria o conteúdo da mensagem de entrada
            content = types.Content(role="user", parts=[types.Part(text=message_text)])

            final_response = ""
            # Itera assincronamente pelos eventos retornados durante a execução do agente
            async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
                if event.is_final_response():
                  for part in event.content.parts:
                    if part.text is not None:
                      final_response += part.text
                      final_response += "\n"
            # Se chegou aqui sem erro, retorna a resposta e sai do loop de retry
            return final_response

        except exceptions.ServerError as e:
            # Verifica se é um erro 503 UNAVAILABLE
            if e.code == 503:
                print(f"Erro 503 UNAVAILABLE na tentativa {attempt + 1}/{max_retries}. Tentando novamente...")
                # Calcula o atraso com exponencial backoff
                delay = initial_delay * (2 ** attempt)
                await asyncio.sleep(delay)
            else:
                # Se for outro ServerError, relança a exceção
                raise e
        except Exception as e:
            # Captura outros erros inesperados e relança
            print(f"Erro inesperado na tentativa {attempt + 1}/{max_retries}: {e}")
            raise e

    # Se todas as tentativas falharam, levanta uma exceção
    raise Exception(f"Falha após {max_retries} tentativas de chamar o agente {agent.name}.")

In [43]:

# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [44]:
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-1.5-flash"

In [45]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [46]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais.
        Você recebe uma lista de lançamentos recentes do agente buscador e,
        - Para cada um dos lançamentos recebidos, você deve usar a ferramenta
        de busca do Google (google_search) para buscar os pontos mais relevantes
        que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais
        informações sobre cada um dos temas e aprofundar.
        - Depois de terminar a busca, você irá escolher APENAS UM tema dentre todos eles,
        aquele tema com potencial de ser o mais relevante com base nas suas pesquisas.
        - Depois de escolher o tema mais relevante dentre todos eles, você irá retornar
        qual foi o tema escolhido, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [47]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO_ROBUSTO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    # Executa o agente
    post_final = await call_agent(redator, entrada_do_agente_redator)
    return post_final


In [48]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")


In [50]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: uso da IA no serviço publico

Maravilha! Vamos então criar o post sobre novidades em uso da IA no serviço publico

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais recentes sobre o uso da IA no serviço público, vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas pesquisas, aqui estão alguns lançamentos e desenvolvimentos notáveis relacionados ao uso de IA no serviço público, focando em notícias de maio de 2025 e meses anteriores:
> 
> 1.  **Plano Brasileiro de Inteligência Artificial (PBIA):** O governo federal está investindo R$ 23 bilhões em quatro anos para transformar o Brasil em referência mundial em inovação e eficiência no uso da IA, especialmente no setor público. O plano visa equipar o país com infraestrutura tecnológica avançada, incluindo supercomputadores alimentados por energias renováveis, e desenvolver processadores de IA de alto desempenho. Um Grupo de Trabalho (GT) foi criado para operacionalizar a gestão do PBIA. (Fonte: Agência Brasil, Portal Gov.br)
> 
> 2.  **IA Generativa no Serviço Público:** O Ministério da Gestão e da Inovação em Serviços Públicos (MGI) e o Serpro lançaram uma cartilha sobre o uso responsável da IA generativa no serviço público. O guia oferece orientações práticas para o uso ético e eficaz dessas ferramentas, com o objetivo de apoiar servidores públicos no entendimento e aplicação da IA. (Fonte: Portal Gov.br, Serpro)
> 
> 3.  **IA GOV Fórum ABEP-TIC:** Um evento foi realizado em São Paulo para debater a aplicação prática de IA na gestão pública. O fórum reuniu autoridades governamentais e representantes de entidades de tecnologia para discutir o futuro da gestão pública com IA. (Fonte: Portal ABEP)
> 
> 4.  **Tramitação de PL para uso de IA no serviço público:** Um projeto de lei (PL 207/2025) que autoriza o uso de tecnologias de inteligência artificial (IA) na prestação de serviços públicos está em tramitação. O projeto prevê a aplicação de IA em áreas como saúde, educação e segurança pública, desde que observados os princípios da ética e respeito aos direitos fundamentais. (Fonte: Portal CMBH)
> 
> 5.  **Tendências de Inovação:** Em 2025, espera-se que a IA conversacional, plataformas integradas e análises preditivas estejam mais presentes na gestão pública. A integração de dados e a interoperabilidade entre sistemas governamentais são consideradas fundamentais para uma administração pública mais eficiente. (Fonte: InBot, MIT Technology Review)
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos e desenvolvimentos recentes relacionados ao uso da IA no serviço público, o tema mais relevante para um post de mídia social é:
> 
> **IA Generativa no Serviço Público: Lançamento de Cartilha do MGI e Serpro**
> 
> **Justificativa:**
> 
> *   **Relevância:** O lançamento de uma cartilha sobre o uso responsável da IA generativa no serviço público é um tema atual e de grande importância. Ele aborda diretamente a necessidade de orientar os servidores públicos sobre como utilizar essas ferramentas de forma ética e eficaz.
> *   **Impacto:** A IA generativa tem o potencial de transformar a maneira como os serviços públicos são prestados, tornando-os mais eficientes e acessíveis. No entanto, é fundamental que seu uso seja feito de forma responsável para evitar riscos e garantir a proteção dos direitos dos cidadãos.
> *   **Engajamento:** O tema é de interesse tanto para servidores públicos quanto para cidadãos em geral, pois afeta a qualidade dos serviços prestados pelo governo. Além disso, a discussão sobre ética e responsabilidade no uso da IA pode gerar debates e reflexões importantes nas redes sociais.
> 
> **Pontos Mais Relevantes a Serem Abordados:**
> 
> *   **O que é IA Generativa:** Explicar de forma simples e didática o que é IA generativa e como ela pode ser utilizada no serviço público.
> *   **Objetivo da Cartilha:** Destacar o objetivo da cartilha lançada pelo MGI e Serpro, que é orientar os servidores públicos sobre o uso responsável da IA generativa.
> *   **Princípios e Diretrizes:** Apresentar os principais princípios e diretrizes contidos na cartilha, como ética, transparência, segurança e respeito aos direitos dos cidadãos.
> *   **Exemplos de Aplicação:** Mostrar exemplos práticos de como a IA generativa pode ser utilizada no serviço público, como na criação de chatbots para atendimento ao cidadão, na geração de relatórios e documentos, e na análise de dados para tomada de decisões.
> *   **Riscos e Desafios:** Alertar sobre os riscos e desafios do uso da IA generativa, como a disseminação de informações falsas, a discriminação algorítmica e a perda de empregos.
> *   **Onde Encontrar a Cartilha:** Informar onde os servidores públicos e demais interessados podem encontrar e acessar a cartilha completa.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Título:** "IA Generativa no Serviço Público: Guia Para Uso Responsável"
> 2.  **Imagem/Vídeo:** Utilizar uma imagem ou vídeo que ilustre o uso da IA generativa no serviço público, como um chatbot atendendo um cidadão ou um servidor público utilizando uma ferramenta de IA para analisar dados.
> 3.  **Texto:**
>     *   **Introdução:** Apresentar o tema da IA generativa e sua importância para o serviço público.
>     *   **Desenvolvimento:** Explicar o objetivo da cartilha lançada pelo MGI e Serpro, destacar os principais princípios e diretrizes, apresentar exemplos de aplicação e alertar sobre os riscos e desafios.
>     *   **Conclusão:** Reforçar a importância do uso responsável da IA generativa e convidar os servidores públicos a acessarem a cartilha completa.
> 4.  **Chamada para Ação:**
>     *   Incentivar os servidores públicos a lerem a cartilha e a aplicarem seus princípios e diretrizes em seu trabalho.
>     *   Convidar os cidadãos a compartilharem suas opiniões e experiências sobre o uso da IA no serviço público.
> 5.  **Hashtags:**
>     *   \#IAGenerativa
>     *   \#ServiçoPúblico
>     *   \#Inovação
>     *   \#Etica
>     *   \#MGI
>     *   \#Serpro
> 
> Com este plano, o post poderá informar, engajar e promover uma discussão relevante sobre o uso da IA generativa no serviço público, contribuindo para uma administração pública mais eficiente, transparente e responsável.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **IA Generativa no Serviço Público: Guia para Uso Responsável!**
> 
> 🚀  O futuro da administração pública está chegando, e ele é inteligente!  Com a força da IA Generativa, podemos revolucionar a forma como os serviços públicos são prestados, tornando-os mais eficientes e acessíveis a todos.
> 
> Mas, atenção!  Usar IA com responsabilidade é crucial.  Por isso, o MGI e o Serpro lançaram uma cartilha incrível que vai te guiar nesse universo!
> 
> ➡️ **Nessa cartilha você encontra:**
> 
> *  **O que é IA Generativa?**  Explicação simples e direta para entender o poder dessa tecnologia.
> *  **Princípios e Diretrizes:**  Ética, transparência e segurança em primeiro lugar!
> *  **Exemplos Práticos:**  Chatbots, análise de dados... veja como a IA já está transformando a rotina pública!
> *  **Riscos e Desafios:**  Vamos falar sobre os desafios para um uso consciente e ético da IA.
> 
> Quer tornar o serviço público ainda melhor?  Então acesse a cartilha completa e faça parte dessa transformação!  Link na bio!
> 
> 
> #IAGenerativa #Serviçopúblico #InovaçãoGov #TransformaçãoDigital
> 


--------------------------------------------------------------


In [52]:
# Cria o diretório 'agentes' se ele não existir
!mkdir -p agentes

In [53]:
%%writefile agentes/redator.py
from google.adk.agents import Agent

def criar_agente_redator(modelo="gemini-1.5-flash"):
    return Agent(
        name="agente_redator",
        model=modelo,
        instruction=\"\"\"
        Você é um redator da Alura. Crie posts para Instagram baseados no plano de conteúdo fornecido.
        Seja criativo, direto, informativo e use hashtags.
        \"\"\",
        description="Cria posts para redes sociais."
    )


Writing agentes/redator.py
